# Getting started with grain

grain is dynamic autograd library on CPU/CUDA devices for D language. While grain's autograd mechanism is dynamic, D compilers can strongly optimize  and validate your scripts statically. This combination gives you great experience.

## how to run grain in jupyter

This installation guide is tested with

- dmd 2.081.2 (recommend [the official installer](https://dlang.org/install.html))
- jupyter 4.4.0 (recommend anaconda)

and install d kernel and engine as follows
``` bash
git clone https://github.com/ShigekiKarita/grain --recursive
cd grain
jupyter kernelspec install ./example/jupyterd --user
dub build
dub build --config=jupyterd --compiler=dmd
export PATH=`pwd`:$PATH
jupyter notebook
```

Then you can create a new notebook with 'd' kernel or open existing 'd' notebooks.

## expected readers

Generally this tutorial is for D users. You can learn D easility in https://tour.dlang.org/tour/ Optionally, experiences of numerical library in D (mir) or in python (numpy, pytorch) help you understanding grain.

## reference

- Main topics in this tutorial are derived from https://pytorch.org/tutorials

## TODO

- DUB extension for drepl like `%load_dub grain`

# mir.ndslice.Slice and numir

Before introducing grain, we will first implement the network using mir. Mir provides an n-dimensional slice value, and many functions for manipulating these slices. Mir is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However we can easily use mir to fit a two-layer network to random data by manually implementing the forward and backward passes through the network using mir operations:

In [1]:
// TODO hand-written 2 layer NN implementation like example/char_rnn_hand.d
import mir.ndslice;

# grain.autograd.Variable

In the above examples, we had to manually implement both the forward and backward passes of our neural network. Manually implementing the backward pass is not a big deal for a small two-layer network, but can quickly get very hairy for large complex networks.

Thankfully, we can use automatic differentiation to automate the computation of backward passes in neural networks. The autograd package in grain provides exactly this functionality. When using autograd, the forward pass of your network will define a computational graph; nodes in the graph will be `Variable`, and edges will be functions that produce output Variable from input Variable. In addition to Slice feature, Variable contains autograd history (simply previously forwarded `Function` object that also implements its backward function) as `grain.autograd.Backprop` type. Backpropagating through this graph then allows you to easily compute gradients. Moreover, unlike mir.ndslice only supports CPU memory slices, grain supports both CPU and CUDA memory slices.

This sounds complicated, it’s pretty simple to use in practice. Each Slice represents a node in a computational graph. If x is a Slice that has x.requiresGrad=True then x.grad is another memory holding the gradient of x with respect to some scalar value (and x.gradSlice hold a slice of the gradient).

Here we use `grain.autograd.Variable` and autograd to implement our two-layer network; now we no longer need to manually implement the backward pass through the network:

In [1]:
// TODO fix empty line outputs
import std.stdio;
import grain;
import mir.random;
import mir.ndslice;
{
    // enable autograd mode
    backprop = true;
    alias ftype = float;
    alias device = HostStorage; // DeviceStorage

    auto learningRate = 1e-6;
    auto batchSize = 6;
    auto inputDim = 10;
    auto hiddenDim = 10;
    auto outputDim = 5;
    
    auto genRand(bool requiresGrad = false, size_t dim)(size_t[dim] s...) {
        return iota(s)
            .map!(_ => rand!ftype)
            .slice
            .variable(requiresGrad)
            .to!device;
    }

    // create random tensors to hold input and outputs
    auto x = genRand(batchSize, inputDim);
    auto y = genRand(batchSize, outputDim);

    auto w1 = genRand!true(inputDim, hiddenDim);
    auto w2 = genRand!true(hiddenDim, outputDim);

    // compose NN sequence like torch with UFCS
    auto y_pred = x
        .matMul(w1)
        .relu
        .matMul(w2);

    auto loss = (y - y_pred).pow(2).sum;
    loss.backward();
    // see backprop start to end
    writeln(loss);
    writeln(w1.gradSlice);
}

Variable!(float, dim=0, HostStorage(T))(data=[33.3019], shape=[], strides=[])
[[2.11398, -2.31107, -3.13537, -3.79735, -3.61724, -2.87242, -0.0406843, -0.448109, 2.69472, 0.494243], [-5.52231, 4.17083, -6.13031, 2.61259, 2.71788, 1.65793, 0.0645454, 3.39182, -5.94427, 2.09116], [-0.0960195, 9.45512, -0.798967, 3.42509, -1.33263, 2.24302, 0.0852675, 1.98408, -1.70982, 2.54118], [4.576, 9.9587, 7.87104, 5.84966, -1.62686, 1.64827, 0.0713128, -3.02025, 2.40107, -1.57483], [-0.376206, -5.97797, 0.38036, -3.24155, -1.86175, -1.587, -0.0498369, 1.05618, 0.545889, 0.704174], [6.30811, 5.44253, -2.19563, 0.304885, 0.0355767, 0.17459, -0.0215853, -2.1838, 0.605075, 1.7944], [-2.37682, -8.08616, 4.39948, 0.659406, 11.2127, 5.55829, -0.063899, 0.444212, -4.53707, -0.319707], [-1.78106, 4.65495, -0.889123, 2.2601, -4.60187, -2.80441, 0.0669973, -0.313492, 0.77061, -1.5902], [4.62098, 1.72052, -4.17155, -2.37387, -5.04137, -3.72468, -0.0394506, -0.85988, 1.14759, 2.15444], [-1.05308, -8.03253, 2.75

In [1]:
// numpy-like broadcasting is available
static assert(!__traits(compiles, [[0.1f], [0.3f]].variable + [0.2f].variable));
// but you need to make them same dimensions
[[0.1f, 0.2f], [0.3f, 0.4f]].variable + [[0.2f]].variable

Variable!(float, dim=2, HostStorage(T))(data=[0.3, 0.4, 0.5, 0.6], shape=[2, 2], strides=[2, 1])


## Defining new autograd functions

Under the hood, each primitive autograd operator is really two functions that operate on Variables. The forward function computes output Variables from input Variable. The backward function receives the gradient of the output Variables with respect to some scalar value, and computes the gradient of the input Variables with respect to that same scalar value.

In grain, we can easily define our own autograd operator by defining a `Function` with `grain.functions.common.FunctionCommon` mixin and implementing the forward and backward functions. We can then use our new autograd operator by constructing an instance and calling it like a function, passing Variables containing input data.

- you can find many examples in `grain.functions` module

In this example we define our own custom autograd function for performing the ReLU nonlinearity, and use it to implement our two-layer network:

In [1]:
/// rectified linear unit nonlinearity
struct MyReLU(T, size_t dim) {

    mixin FunctionCommon;
    Variable!(T, dim, HostStorage) hx;

    auto forward(Variable!(T, dim, HostStorage) x) {
        import mir.ndslice : each;
        this.hx = x.dup;
        auto y = x.dup;
        y.sliced.each!((ref a) {
            if (a < 0)
                a = 0;
        });
        return y;
    }

    auto backward(Variable!(T, dim, HostStorage) gy) {
        auto gx = gy.dup;
        foreach (i; 0 .. gx.data.length) {
            if (this.hx.data[i] < 0.0)
                gx.data[i] = 0.0;
        }
        return gx;
    }
}